In [ ]:
!pip install kagglehub

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
import kagglehub
path = kagglehub.dataset_download("naveedhn/amazon-product-review-spam-and-non-spam")
print("Path to dataset files:", path)

100%|██████████| 5.11G/5.11G [00:48<00:00, 114MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1


# **Converting the json files to csv file**

In [ ]:
!ls /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1


Cell_Phones_and_Accessories  Home_and_Kitchen  Sports_and_Outdoors
Clothing_Shoes_and_Jewelry   part.json	       Toys_and_Games
Electronics		     separate.json


In [ ]:
!pip install ijson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 5.9 MB/s eta 0:00:00


In [ ]:
import os, csv, ijson

base_folder = "/root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1"
output_csv = "/content/amazon_all.csv"

fieldnames = set(["category"])
total_records = 0
broken_files = []

def fix_json_structure(base_folder, output_path):
    with open(base_folder, 'r') as infile, open(output_path, 'w') as outfile:
        outfile.write("[")
        first = True
        for line in infile:
            line = line.strip()
            if not line:
                continue
            if not line.startswith("{"):
                continue
            if not first:
                outfile.write(",")
            outfile.write(line)
            first = False
        outfile.write("]")

for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith(".json"):
            json_path = os.path.join(root, file)
            print(f"Check columns from: {json_path}")

            fixed_json = "fixed_temp.json"
            fix_json_structure(json_path, fixed_json)

            try:
                with open(fixed_json, "r") as jf:
                    for record in ijson.items(jf, "item"):
                        fieldnames.update(record.keys())
            except Exception as e:
                print(f"File skipped due to an error: {file} | {e}")
                broken_files.append(file)

print(f"\n {len(fieldnames)} columns discovered (including 'category').\n")

with open(output_csv, "w", newline="", encoding="utf-8") as cf:
    writer = csv.DictWriter(cf, fieldnames=list(fieldnames))
    writer.writeheader()

    for root, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith(".json"):
                json_path = os.path.join(root, file)
                category_name = os.path.basename(root)
                print(f"Translating: {json_path} | category: {category_name}")

                fixed_json = "fixed_temp.json"
                fix_json_structure(json_path, fixed_json)

                try:
                    with open(fixed_json, "r") as jf:
                        for record in ijson.items(jf, "item"):
                            record["category"] = category_name
                            writer.writerow(record)
                            total_records += 1
                except Exception as e:
                    print(f"Skipped file while writing: {file} | {e}")
                    broken_files.append(file)

print(f"\n Converted to: {output_csv}")
print(f"Number of written records: {total_records:,}")

if broken_files:
    print("\n Files with errors that were not converted:")
    for f in broken_files:
        print("   -", f)


Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/part.json/part.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/Toys_and_Games/Toys_and_Games.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/Sports_and_Outdoors/Sports_and_Outdoors.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/Electronics/Electronics.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spam/versions/1/Home_and_Kitchen/Home_and_Kitchen.json
Check columns from: /root/.cache/kagglehub/datasets/naveedhn/amazon-product-review-spam-and-non-spa

In [ ]:
import os

file_path = "/content/amazon_all.csv"
file_size_gb = os.path.getsize(file_path) / (1024 ** 3)

print(f"Size of file: {file_size_gb:.2f} GB")

Size of file: 12.60 GB


# **First method "using pandas.read_csv(chunk size)**

In [ ]:
import pandas as pd
import time
import psutil

csv_path = r"/content/amazon_all.csv"
chunksize = 200000
process = psutil.Process()
memory_before = process.memory_info().rss / (1024 ** 2)
start_time = time.time()
total_rows = 0
first_chunk = None
max_memory = 0

for chunk in pd.read_csv(csv_path, chunksize=chunksize):
    total_rows += len(chunk)

    if first_chunk is None:
        first_chunk = chunk

    memory = chunk.memory_usage(deep=True).sum() / (1024**2)
    if memory > max_memory:
        max_memory = memory

end_time = time.time()
memory_after = process.memory_info().rss / (1024 ** 2)
memory_used = memory_after - memory_before

print(f"The time taken to read the file is: {end_time - start_time:.2f} seconds")
print(f"Total rows read: {total_rows}")
print(f"Maximum memory used by a chunk: {max_memory:.2f} MB")
print(f"Memory used by using pandas: {memory_used:.2f} MB")

first_chunk.head()

/tmp/ipython-input-3849576156.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_path, chunksize=chunksize):
/tmp/ipython-input-3849576156.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_path, chunksize=chunksize):


The time taken to read the file is: 652.06 seconds
Total rows read: 25363163
Maximum memory used by a chunk: 227.99 MB
Memory used by using pandas: 587.36 MB


,_id,label,reviewerName,asin,summary,reviewTime,BehaviouralFeatureResult,unixReviewTime,category,reviewText,reviewerID,helpful,class,overall
0,{'$oid': '5a421d732bfbb88f180e24d2'},0.0,Barry,B000809PCG,Not the first game I reach for...,"01 9, 2013","{'CS': Decimal('0.525950681'), 'MNR': Decimal(...",1357689600,part.json,"it's ok, but not great fun. maybe it just mak...",A3ITLRYWLKJP5B,"[Decimal('1.0'), Decimal('2.0')]",NaN,3.0
1,{'$oid': '5a421d732bfbb88f180e24d3'},0.0,J Ayles,B004Y6F9KW,Cool,"08 25, 2011","{'CS': Decimal('0.0'), 'MNR': Decimal('0.0'), ...",1314230400,part.json,I love this Air-soft rifle. I love the scope i...,AA65S7IF7V9GU,"[Decimal('0.0'), Decimal('0.0')]",NaN,5.0
2,{'$oid': '5a421d732bfbb88f180e24d4'},0.0,"Helene M. Jolley ""Shopaholic""",B0041RR5X8,Flimsy but serviceable,"07 8, 2012","{'CS': Decimal('0.639148171'), 'MNR': Decimal(...",1341705600,part.json,Bought this for my new bathroom and though it ...,A3EBF7VUVF09GW,"[Decimal('0.0'), Decimal('0.0')]",NaN,3.0
3,{'$oid': '5a421d732bfbb88f180e24d5'},0.0,amazonfan111,B00066D6KO,"""Medium weight"" = very light","08 24, 2013","{'CS': Decimal('0.360997436'), 'MNR': Decimal(...",1377302400,part.json,"I guess the description of ""4 lbs of force nee...",A14YSDP7TH705X,"[Decimal('0.0'), Decimal('0.0')]",NaN,3.0
4,{'$oid': '5a421d732bfbb88f180e24d6'},0.0,sherri,B0057MGKO6,Only received 5 of the 10! What's up with that...,"01 28, 2014","{'CS': Decimal('0.0'), 'MNR': Decimal('0.0'), ...",1390867200,part.json,Only receive 5 of the 10? What's up with that?...,A2VNHQNN1T3KKN,"[Decimal('0.0'), Decimal('0.0')]",NaN,1.0


# **Second method "using Dask"**

In [ ]:
import dask.dataframe as dd

start_time = time.time()
process = psutil.Process()
memory_before = process.memory_info().rss / (1024 ** 2)

df = dd.read_csv("/content/amazon_all.csv", blocksize="250MB", dtype={'asin': 'object', 'BehaviouralFeatureResult': 'object'})

total_rows = df.shape[0].compute()
memory_after = process.memory_info().rss / (1024 ** 2)
end_time = time.time()

print(f"The time taken to read file with Dask: {end_time - start_time:.2f} seconds")
print(f"Memory used by Dask: {memory_after - memory_before:.2f} MB")
print(f"Total rows: {total_rows:,}")

The time taken to read file with Dask: 542.68 seconds
Memory used by Dask: 863.29 MB
Total rows: 25,363,163


# **Third method "with Compression"**

In [ ]:
import gzip
import shutil

with open("/content/amazon_all.csv", 'rb') as f_in:
    with gzip.open("/content/amazon_all.csv.gz", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
process = psutil.Process()

memory_before = process.memory_info().rss / (1024 ** 2)
start_time = time.time()
df = pd.read_csv("/content/amazon_all.csv.gz", compression='gzip')
end_time = time.time()
memory_after = process.memory_info().rss / (1024 ** 2)
memory_used = memory_after - memory_before
print(f"The time taken to read compressed CSV (gzip): {end_time - start_time:.2f} seconds")
print(f"Total rows read: {len(df):,}")
print(f"Memory used for reading compressed file: {memory_used:.2f} MB")
df.head()

In [ ]:
chunksize = 200000
process = psutil.Process()
memory_before = process.memory_info().rss / (1024 ** 2)

start_time = time.time()
csv_iterator = pd.read_csv("/content/amazon_all.csv.gz", compression='gzip', chunksize=chunksize)
total_rows = 0
first_chunk = None
max_memory = 0

for chunk in csv_iterator:
    total_rows += len(chunk)

    if first_chunk is None:
        first_chunk = chunk

    memory = chunk.memory_usage(deep=True).sum() / (1024 ** 2)
    if memory > max_memory:
        max_memory = memory

end_time = time.time()
memory_after = process.memory_info().rss / (1024 ** 2)
memory_used = memory_after - memory_before

print(f"The time taken to read compressed CSV (gzip): {end_time - start_time:.2f} seconds")
print(f"Total rows read: {total_rows:,}")
print(f"Maximum memory used by a chunk: {max_memory:.2f} MB")
print(f"Memory used by using compression file: {memory_used:.2f} MB")
first_chunk.head()

/tmp/ipython-input-2700501805.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in csv_iterator:
/tmp/ipython-input-2700501805.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in csv_iterator:


The time taken to read compressed CSV (gzip): 760.83 seconds
Total rows read: 25,363,163
Maximum memory used by a chunk: 227.99 MB
Memory used by using compression file: 623.32 MB


,_id,label,reviewerName,asin,summary,reviewTime,BehaviouralFeatureResult,unixReviewTime,category,reviewText,reviewerID,helpful,class,overall
0,{'$oid': '5a421d732bfbb88f180e24d2'},0.0,Barry,B000809PCG,Not the first game I reach for...,"01 9, 2013","{'CS': Decimal('0.525950681'), 'MNR': Decimal(...",1357689600,part.json,"it's ok, but not great fun. maybe it just mak...",A3ITLRYWLKJP5B,"[Decimal('1.0'), Decimal('2.0')]",NaN,3.0
1,{'$oid': '5a421d732bfbb88f180e24d3'},0.0,J Ayles,B004Y6F9KW,Cool,"08 25, 2011","{'CS': Decimal('0.0'), 'MNR': Decimal('0.0'), ...",1314230400,part.json,I love this Air-soft rifle. I love the scope i...,AA65S7IF7V9GU,"[Decimal('0.0'), Decimal('0.0')]",NaN,5.0
2,{'$oid': '5a421d732bfbb88f180e24d4'},0.0,"Helene M. Jolley ""Shopaholic""",B0041RR5X8,Flimsy but serviceable,"07 8, 2012","{'CS': Decimal('0.639148171'), 'MNR': Decimal(...",1341705600,part.json,Bought this for my new bathroom and though it ...,A3EBF7VUVF09GW,"[Decimal('0.0'), Decimal('0.0')]",NaN,3.0
3,{'$oid': '5a421d732bfbb88f180e24d5'},0.0,amazonfan111,B00066D6KO,"""Medium weight"" = very light","08 24, 2013","{'CS': Decimal('0.360997436'), 'MNR': Decimal(...",1377302400,part.json,"I guess the description of ""4 lbs of force nee...",A14YSDP7TH705X,"[Decimal('0.0'), Decimal('0.0')]",NaN,3.0
4,{'$oid': '5a421d732bfbb88f180e24d6'},0.0,sherri,B0057MGKO6,Only received 5 of the 10! What's up with that...,"01 28, 2014","{'CS': Decimal('0.0'), 'MNR': Decimal('0.0'), ...",1390867200,part.json,Only receive 5 of the 10? What's up with that?...,A2VNHQNN1T3KKN,"[Decimal('0.0'), Decimal('0.0')]",NaN,1.0


In [ ]:
data = {
    "Method": ["Pandas", "Dask", "Compressed (Gzip)"],
    "Time (seconds)": [652.06, 542.68, 760.83],
    "Max Memory per Chunk (MB)": [227.99, None, 227.99],
    "Total Memory Used (MB)": [587.36, 863.29, 623.32]
}
df_comparison = pd.DataFrame(data)
print("\nThe Comparison:\n")
print(df_comparison.to_string(index=False))



The Comparison:

           Method  Time (seconds)  Max Memory per Chunk (MB)  Total Memory Used (MB)
           Pandas          652.06                     227.99                  587.36
             Dask          542.68                        NaN                  863.29
Compressed (Gzip)          760.83                     227.99                  623.32
